/Users/kevinbrundler/Desktop/ATAI/movie-bot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.24s/it]


In [9]:
query = "When was Angelina Jolie born?"
columns = ['placeofbirth', 'significantevent', 'ancestralhome']
prompt = f"Query: {query}\n\nColumns:\n{', '.join(columns)}\n\nWhich columns are relevant to answer the query?"

inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=30)

relevant_columns = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Relevant Columns:", relevant_columns)

Relevant Columns: Query: When was Angelina Jolie born?

Columns:
placeofbirth, significantevent, ancestralhome

Which columns are relevant to answer the query?

Response:The relevant columns to answer the query about Angelina Jolie's birth date are:

- placeofbirth


In [31]:
import time

def execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time() 
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_duration = end_time - start_time
        print(f"Execution time for {func.__name__}: {execution_duration:.4f} seconds")
        return result
    return wrapper

In [32]:
def generate_sparql(query: str) -> str:
    """
    Takes a natural language query and converts it to a SPARQL query using GPT-J-6B.
    """
    
    schema_ddis = "<http://ddis.ch/atai/>"
    schema_wd = "<http://www.wikidata.org/entity/>"
    schema_wdt = "<http://www.wikidata.org/prop/direct/>"
    schema_schema = "<http://schema.org/>"
    
    # Format the prompt to help guide the LLM
    
    # Tokenize input
    inputs = tokenizer(query, return_tensors="pt")
    
    # Generate SPARQL query
    outputs = model.generate(
        inputs.input_ids,
        max_length=50,  # Adjust based on expected query length
        num_beams=5,     # Beam search to improve output quality
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    
    # Decode and clean up output
    generated_sparql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Post-processing: Extract the generated SPARQL part
    sparql_start = "The SPARQL query is:"
    if sparql_start in generated_sparql: generated_sparql = generated_sparql.split(sparql_start)[-1].strip()
    
    return generated_sparql

In [33]:
@execution_time
def generate_answer(query, information):
    try:
        system_prompt = (
            "You are a knowledgeable assistant specializing in movies and well-known individuals. Your task is to answer "
            "the following question as accurately and concise as possible, based on the provided context."
        )
                
        context_parts = [system_prompt, f"Question: {query}\n\n", "Retrieved Information:\n\n"]
        
        for entity_id, data in information.items():
            entity_name = data.get('entity_name', 'Unknown')
            context_parts.append(f"Entity: {entity_name} ({entity_id})\n")
            for prop, values in data.items():
                values = values if isinstance(values, list) else [values]
                val_str = ', '.join(values)
                context_parts.append(f" - {prop}: {val_str}\n")
            context_parts.append("\n")
        
        context_parts.append("Answer:")
        context_str = ''.join(context_parts)
        
        inputs = tokenizer(context_str, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                max_new_tokens = 70,
                num_return_sequences=1,       # Single answer
                temperature=1.0,              # Lower temperature for more focused answers
                top_p=0.9                     # Nucleus sampling for a more diverse yet concise response
            )
        
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = answer.split("Answer:")[-1].strip()
        return answer
    except Exception as e:
        print(f"Error generating answer: {e}")
        return "I'm sorry, I encountered an error while processing your request."

In [34]:
generate_sparql("Who directed Inception?")

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


'Who directed Inception?\nInception, directed by Christopher Nolan, is a 2010 science fiction heist thriller film that explores the concept of shared dreaming and the manipulation of the subconscious mind. The'